In [1]:
# Description

# Requirements
import os
import geopandas as gpd
import pandas as pd
import pathlib
from pathlib import Path
import json
from collections import defaultdict
import fiona


# Parameters
data_foldername = 'data'
data_directory = pathlib.Path('.').absolute() / data_foldername
data_directory = str(data_directory)
data_directory = data_directory + '\\'

gpx_foldername = 'gpxfiles'
gpx_directory = pathlib.Path('.').absolute() / gpx_foldername

#new_gpx_foldername = 'new_gpxfiles'
#gpx_directory = pathlib.Path('.').absolute() / new_gpx_foldername 

#processed_gpx_foldername = 'processed_gpxfiles'
#processed_gpx_directory = pathlib.Path('.').absolute() / processed_gpx_foldername 

# List of GeoPackage files to read
gpkg_files = os.listdir(data_directory)

In [2]:
## Proces for admin regions 0 (country-level)

# Initialize an empty list to store GeoDataFrames
gdf_list = []

# Loop through each GeoPackage file and read the data
for gpkg_file in gpkg_files:
    # Read the GeoPackage file
    gdf_file = gpd.read_file(os.path.join(data_directory,  gpkg_file), layer='ADM_ADM_0')

    gdf_list.append(gdf_file)

gdf_adm0 = pd.concat(gdf_list, ignore_index=True)

# Reset the index of the merged GeoDataFrame
gdf_adm0 = gdf_adm0.reset_index(drop=True)

#gdf_adm0.explore()

# Initialize an empty list to store GeoDataFrames
gdf_adminregions_per_gpx = []

# Proces per gpx-file
#for filename in os.listdir(gpx_directory):
#    if filename.endswith('.gpx'):
#        gpxfile = os.path.join(gpx_directory, filename)
#        gpx = gpd.read_file(gpxfile, layer='tracks')
#        gdf_gpxline = gpx[gpx['name'].str.contains("Running")]
        
        # Perform the intersection
#        gdf_join = gpd.sjoin(left_df=gdf_adm0, right_df=gdf_gpxline,  how="inner", predicate="intersects")
        #print(gdf_join)
        
        #Append data to temp gdf
#        gdf_adminregions_per_gpx.append(gdf_join)

# Create gdf with information from all gpx-files
#gdf_runs_adm0 = gpd.GeoDataFrame( pd.concat(gdf_adminregions_per_gpx, ignore_index=True) )

# Initialize a DataFrame to keep track of stats
admin_region_stats = pd.DataFrame(columns=['admin_id', 'count', 'first_date', 'last_date'])

for filename in os.listdir(gpx_directory):
    if filename.endswith('.gpx'):
        gpxfile = os.path.join(gpx_directory, filename)
        gpx = gpd.read_file(gpxfile, layer='tracks')
        gdf_gpxline = gpx[gpx['name'].str.contains("Running")]
        date = filename[:10]
        gdf_gpxline['date'] = date
        run_date = date

        # Perform the intersection
        gdf_join = gpd.sjoin(left_df=gdf_adm0, right_df=gdf_gpxline, how="inner", predicate="intersects")
        # Get unique admin regions in this intersection
        admin_ids = gdf_join['GID_0'].unique()

        # Update stats for each admin region
        for admin_id in admin_ids:
            if admin_id in admin_region_stats['admin_id'].values:
                # Update count
                admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'count'] += 1
                # Update first_date if current run_date is earlier or if first_date is None
                if run_date:
                    current_first_date = admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'first_date'].iloc[0]
                    if current_first_date is None or run_date < current_first_date:
                        admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'first_date'] = run_date
                    # Update last_date if current run_date is later or if last_date is None
                    current_last_date = admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'last_date'].iloc[0]
                    if current_last_date is None or run_date > current_last_date:
                        admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'last_date'] = run_date
            else:
                # Add new entry
                admin_region_stats.loc[len(admin_region_stats)] = [admin_id, 1, run_date, run_date]

        # Append data to temp gdf
        gdf_adminregions_per_gpx.append(gdf_join)


# Create gdf with information from all gpx-files
gdf_runs_adm0 = gpd.GeoDataFrame(pd.concat(gdf_adminregions_per_gpx, ignore_index=True))

# Merge stats back into the main GeoDataFrame
gdf_runs_adm0 = gdf_runs_adm0.merge(
    admin_region_stats,
    left_on='GID_0',
    right_on='admin_id',
    how='left'
)

# Select columns
#col_list = ['GID_0', 'COUNTRY', 'geometry']
col_list = ['COUNTRY', 'count', 'first_date', 'last_date', 'geometry']
gdf_runs_adm0 = gdf_runs_adm0[col_list]
gdf_runs_adm0 = gdf_runs_adm0.rename(columns={"COUNTRY": "NAME"})
gdf_runs_adm0.columns = gdf_runs_adm0.columns.str.lower()

gdf_runs_adm0 = gdf_runs_adm0.drop_duplicates(["geometry"])

# Simplify geometries with a tolerance (in units of the CRS)
# The higher the tolerance, the more simplified the geometry
tolerance = 0.001  # Adjust as needed
gdf_runs_adm0['geometry'] = gdf_runs_adm0['geometry'].simplify(tolerance)

# Write geojsonfile
if(os.path.isfile("adm_0.geojson")):
    os.remove("adm_0.geojson")
    print("File Deleted successfully")
else:
    print("File does not exist")

gdf_runs_adm0.to_file("adm_0.geojson", driver='GeoJSON')

#gdf_runs_adm0.explore()
#gdf_runs_adm0.head()

File Deleted successfully


In [3]:
## Proces for admin regions 1 (province-level)

# Initialize an empty list to store GeoDataFrames
gdf_list = []

# Loop through each GeoPackage file and read the data
for gpkg_file in gpkg_files:
    # Read the GeoPackage file
    gdf_file = gpd.read_file(os.path.join(data_directory,  gpkg_file), layer='ADM_ADM_1')
    gdf_list.append(gdf_file)

gdf_adm1 = pd.concat(gdf_list, ignore_index=True)

gdf_adm1 = gdf_adm1[~gdf_adm1['ENGTYPE_1'].str.contains("Water body")]

# Reset the index of the merged GeoDataFrame
gdf_adm1 = gdf_adm1.reset_index(drop=True)

# Initialize an empty list to store GeoDataFrames
gdf_adminregions_per_gpx = []

# Proces per gpx-file
#for filename in os.listdir(gpx_directory):
#    if filename.endswith('.gpx'):
#        gpxfile = os.path.join(gpx_directory, filename)
#        gpx = gpd.read_file(gpxfile, layer='tracks')
#        gdf_gpxline = gpx[gpx['name'].str.contains("Running")]

        # Perform the intersection
#        gdf_join = gpd.sjoin(left_df=gdf_adm1, right_df=gdf_gpxline,  how="inner", predicate="intersects")
        #print(gdf_join)
        
        #Append data to temp gdf
#        gdf_adminregions_per_gpx.append(gdf_join)

# Initialize a DataFrame to keep track of stats
admin_region_stats = pd.DataFrame(columns=['admin_id', 'count', 'first_date', 'last_date'])

for filename in os.listdir(gpx_directory):
    if filename.endswith('.gpx'):
        gpxfile = os.path.join(gpx_directory, filename)
        gpx = gpd.read_file(gpxfile, layer='tracks')
        gdf_gpxline = gpx[gpx['name'].str.contains("Running")]
        date = filename[:10]
        gdf_gpxline['date'] = date
        run_date = date

        # Perform the intersection
        gdf_join = gpd.sjoin(left_df=gdf_adm1, right_df=gdf_gpxline, how="inner", predicate="intersects")
        # Get unique admin regions in this intersection
        admin_ids = gdf_join['GID_1'].unique()

        # Update stats for each admin region
        for admin_id in admin_ids:
            if admin_id in admin_region_stats['admin_id'].values:
                # Update count
                admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'count'] += 1
                # Update first_date if current run_date is earlier or if first_date is None
                if run_date:
                    current_first_date = admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'first_date'].iloc[0]
                    if current_first_date is None or run_date < current_first_date:
                        admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'first_date'] = run_date
                    # Update last_date if current run_date is later or if last_date is None
                    current_last_date = admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'last_date'].iloc[0]
                    if current_last_date is None or run_date > current_last_date:
                        admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'last_date'] = run_date
            else:
                # Add new entry
                admin_region_stats.loc[len(admin_region_stats)] = [admin_id, 1, run_date, run_date]

        # Append data to temp gdf
        gdf_adminregions_per_gpx.append(gdf_join)


# Create gdf with information from all gpx-files
gdf_runs_adm1 = gpd.GeoDataFrame( pd.concat(gdf_adminregions_per_gpx, ignore_index=True) )

# Merge stats back into the main GeoDataFrame
gdf_runs_adm1 = gdf_runs_adm1.merge(
    admin_region_stats,
    left_on='GID_1',
    right_on='admin_id',
    how='left'
)

# Select columns
col_list = ['COUNTRY', 'NAME_1', 'TYPE_1', 'count', 'first_date', 'last_date', 'geometry']
gdf_runs_adm1 = gdf_runs_adm1[col_list]
gdf_runs_adm1 = gdf_runs_adm1.rename(columns={"NAME_1": "NAME"})
gdf_runs_adm1 = gdf_runs_adm1.rename(columns={"TYPE_1": "TYPE"})
gdf_runs_adm1.columns = gdf_runs_adm1.columns.str.lower()

gdf_runs_adm1 = gdf_runs_adm1.drop_duplicates(["geometry"])

# Simplify geometries with a tolerance (in units of the CRS)
# The higher the tolerance, the more simplified the geometry
tolerance = 0.001  # Adjust as needed
gdf_runs_adm1['geometry'] = gdf_runs_adm1['geometry'].simplify(tolerance)

# Write geojsonfile
if(os.path.isfile("adm_1.geojson")):
    os.remove("adm_1.geojson")
    print("File Deleted successfully")
else:
    print("File does not exist")

gdf_runs_adm1.to_file("adm_1.geojson", driver='GeoJSON')

File Deleted successfully


In [4]:
## Proces for admin regions 2 (municipality-level)

# Initialize an empty list to store GeoDataFrames
gdf_list = []

# Loop through each GeoPackage file and read the data
for gpkg_file in gpkg_files:
    # Read the GeoPackage file
    gdf_file = gpd.read_file(os.path.join(data_directory,  gpkg_file), layer='ADM_ADM_2')
    gdf_list.append(gdf_file)

gdf_adm2 = pd.concat(gdf_list, ignore_index=True)

gdf_adm2 = gdf_adm2[~gdf_adm2['ENGTYPE_2'].str.contains("Water body")]

# Reset the index of the merged GeoDataFrame
gdf_adm2 = gdf_adm2.reset_index(drop=True)

# Initialize an empty list to store GeoDataFrames
gdf_adminregions_per_gpx = []

# Proces per gpx-file
#for filename in os.listdir(gpx_directory):
#    if filename.endswith('.gpx'):
#        gpxfile = os.path.join(gpx_directory, filename)
#        gpx = gpd.read_file(gpxfile, layer='tracks')
#        gdf_gpxline = gpx[gpx['name'].str.contains("Running")]
        
        # Perform the intersection
#        gdf_join = gpd.sjoin(left_df=gdf_adm2, right_df=gdf_gpxline,  how="inner", predicate="intersects")
        #print(gdf_join)
        
        #Append data to temp gdf
#        gdf_adminregions_per_gpx.append(gdf_join)

# Initialize a DataFrame to keep track of stats
admin_region_stats = pd.DataFrame(columns=['admin_id', 'count', 'first_date', 'last_date'])

for filename in os.listdir(gpx_directory):
    if filename.endswith('.gpx'):
        gpxfile = os.path.join(gpx_directory, filename)
        gpx = gpd.read_file(gpxfile, layer='tracks')
        gdf_gpxline = gpx[gpx['name'].str.contains("Running")]
        date = filename[:10]
        gdf_gpxline['date'] = date
        run_date = date

        # Perform the intersection
        gdf_join = gpd.sjoin(left_df=gdf_adm2, right_df=gdf_gpxline, how="inner", predicate="intersects")
        # Get unique admin regions in this intersection
        admin_ids = gdf_join['GID_2'].unique()

        # Update stats for each admin region
        for admin_id in admin_ids:
            if admin_id in admin_region_stats['admin_id'].values:
                # Update count
                admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'count'] += 1
                # Update first_date if current run_date is earlier or if first_date is None
                if run_date:
                    current_first_date = admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'first_date'].iloc[0]
                    if current_first_date is None or run_date < current_first_date:
                        admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'first_date'] = run_date
                    # Update last_date if current run_date is later or if last_date is None
                    current_last_date = admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'last_date'].iloc[0]
                    if current_last_date is None or run_date > current_last_date:
                        admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'last_date'] = run_date
            else:
                # Add new entry
                admin_region_stats.loc[len(admin_region_stats)] = [admin_id, 1, run_date, run_date]

        # Append data to temp gdf
        gdf_adminregions_per_gpx.append(gdf_join)

# Create gdf with information from all gpx-files
gdf_runs_adm2 = gpd.GeoDataFrame( pd.concat(gdf_adminregions_per_gpx, ignore_index=True) )

# Merge stats back into the main GeoDataFrame
gdf_runs_adm2 = gdf_runs_adm2.merge(
    admin_region_stats,
    left_on='GID_2',
    right_on='admin_id',
    how='left'
)

# Select columns
col_list = ['COUNTRY', 'NAME_2', 'TYPE_2', 'count', 'first_date', 'last_date', 'geometry']
gdf_runs_adm2 = gdf_runs_adm2[col_list]
gdf_runs_adm2 = gdf_runs_adm2.rename(columns={"NAME_2": "NAME"})
gdf_runs_adm2 = gdf_runs_adm2.rename(columns={"TYPE_2": "TYPE"})
gdf_runs_adm2.columns = gdf_runs_adm2.columns.str.lower()

gdf_runs_adm2 = gdf_runs_adm2.drop_duplicates(["geometry"])

# Simplify geometries with a tolerance (in units of the CRS)
# The higher the tolerance, the more simplified the geometry
tolerance = 0.001  # Adjust as needed
gdf_runs_adm2['geometry'] = gdf_runs_adm2['geometry'].simplify(tolerance)

# Write geojsonfile
if(os.path.isfile("adm_2.geojson")):
    os.remove("adm_2.geojson")
    print("File Deleted successfully")
else:
    print("File does not exist")

gdf_runs_adm2.to_file("adm_2.geojson", driver='GeoJSON')


File Deleted successfully


In [5]:
## Proces for admin regions 3 (district-level)

# Initialize an empty list to store GeoDataFrames
gdf_list = []

# Specify the layer names you want to check
ADM_ADM_3 = 'ADM_ADM_3'
ADM_ADM_2 = 'ADM_ADM_2'

# Loop through each GeoPackage file and read the data
for gpkg_file in gpkg_files:
    file_path = os.path.join(data_directory, gpkg_file)
    try:
        # Try to read ADM3 layer
        gdf = gpd.read_file(file_path, layer=ADM_ADM_3)
        gdf = gdf[~gdf['ENGTYPE_3'].str.contains("Water body")]
        gdf_list.append(gdf)
        print(f"Layer '{ADM_ADM_3}' read from '{gpkg_file}'.")
    except Exception as e:
        try:
            # Fall back to ADM2 layer
            gdf = gpd.read_file(file_path, layer=ADM_ADM_2)
            gdf = gdf[~gdf['ENGTYPE_2'].str.contains("Water body")]
            gdf_list.append(gdf)
            print(f"Layer '{ADM_ADM_2}' read from '{gpkg_file}' (fallback).")
        except Exception as e:
            print(f"Could not read either '{ADM_ADM_3}' or '{ADM_ADM_2}' from '{gpkg_file}': {e}")

# Concatenate the GeoDataFrames if any were found
if gdf_list:
    gdf_adm3 = pd.concat(gdf_list, ignore_index=True)
    print("Combined GeoDataFrame created.")
else:
    print(f"No GeoPackages contained the layer '{ADM_ADM_3}'.")

#gdf_adm3 = gdf_adm3[~gdf_adm3['ENGTYPE_3'].str.contains("Water body")]

# Reset the index of the merged GeoDataFrame
gdf_adm3 = gdf_adm3.reset_index(drop=True)

# Initialize an empty list to store GeoDataFrames
gdf_adminregions_per_gpx = []

# Proces per gpx-file
#for filename in os.listdir(gpx_directory):
#    if filename.endswith('.gpx'):
#        gpxfile = os.path.join(gpx_directory, filename)
#        gpx = gpd.read_file(gpxfile, layer='tracks')
#        gdf_gpxline = gpx[gpx['name'].str.contains("Running")]
        
        # Perform the intersection
#        gdf_join = gpd.sjoin(left_df=gdf_adm3, right_df=gdf_gpxline,  how="inner", predicate="intersects")
        #print(gdf_join)
        
        #Append data to temp gdf
#        gdf_adminregions_per_gpx.append(gdf_join)

# Initialize a DataFrame to keep track of stats
admin_region_stats = pd.DataFrame(columns=['admin_id', 'count', 'first_date', 'last_date'])

for filename in os.listdir(gpx_directory):
    if filename.endswith('.gpx'):
        gpxfile = os.path.join(gpx_directory, filename)
        gpx = gpd.read_file(gpxfile, layer='tracks')
        gdf_gpxline = gpx[gpx['name'].str.contains("Running")]
        date = filename[:10]
        gdf_gpxline['date'] = date
        run_date = date

        # Perform the intersection
        gdf_join = gpd.sjoin(left_df=gdf_adm3, right_df=gdf_gpxline, how="inner", predicate="intersects")
        # Get unique admin regions in this intersection
        admin_ids = gdf_join['GID_3'].unique()

        # Update stats for each admin region
        for admin_id in admin_ids:
            if admin_id in admin_region_stats['admin_id'].values:
                # Update count
                admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'count'] += 1
                # Update first_date if current run_date is earlier or if first_date is None
                if run_date:
                    current_first_date = admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'first_date'].iloc[0]
                    if current_first_date is None or run_date < current_first_date:
                        admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'first_date'] = run_date
                    # Update last_date if current run_date is later or if last_date is None
                    current_last_date = admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'last_date'].iloc[0]
                    if current_last_date is None or run_date > current_last_date:
                        admin_region_stats.loc[admin_region_stats['admin_id'] == admin_id, 'last_date'] = run_date
            else:
                # Add new entry
                admin_region_stats.loc[len(admin_region_stats)] = [admin_id, 1, run_date, run_date]

        # Append data to temp gdf
        gdf_adminregions_per_gpx.append(gdf_join)

# Create gdf with information from all gpx-files
gdf_runs_adm3 = gpd.GeoDataFrame( pd.concat(gdf_adminregions_per_gpx, ignore_index=True) )

# Merge stats back into the main GeoDataFrame
gdf_runs_adm3 = gdf_runs_adm3.merge(
    admin_region_stats,
    left_on='GID_3',
    right_on='admin_id',
    how='left'
)

# Select columns
col_list = ['COUNTRY', 'NAME_3', 'TYPE_3', 'count', 'first_date', 'last_date', 'geometry']
gdf_runs_adm3 = gdf_runs_adm3[col_list]
gdf_runs_adm3 = gdf_runs_adm3.rename(columns={"NAME_3": "NAME"})
gdf_runs_adm3 = gdf_runs_adm3.rename(columns={"TYPE_3": "TYPE"})
gdf_runs_adm3.columns = gdf_runs_adm3.columns.str.lower()

gdf_runs_adm3 = gdf_runs_adm3.drop_duplicates(["geometry"])

# Simplify geometries with a tolerance (in units of the CRS)
# The higher the tolerance, the more simplified the geometry
tolerance = 0.001  # Adjust as needed
gdf_runs_adm3['geometry'] = gdf_runs_adm3['geometry'].simplify(tolerance)

# Write geojsonfile
if(os.path.isfile("adm_3.geojson")):
    os.remove("adm_3.geojson")
    print("File Deleted successfully")
else:
    print("File does not exist")

gdf_runs_adm3.to_file("adm_3.geojson", driver='GeoJSON')

Layer 'ADM_ADM_3' read from 'gadm41_AUT.gpkg'.
Layer 'ADM_ADM_3' read from 'gadm41_BEL.gpkg'.
Layer 'ADM_ADM_3' read from 'gadm41_CHE.gpkg'.
Layer 'ADM_ADM_2' read from 'gadm41_CZE.gpkg' (fallback).
Layer 'ADM_ADM_3' read from 'gadm41_DEU.gpkg'.
Layer 'ADM_ADM_2' read from 'gadm41_DNK.gpkg' (fallback).
Layer 'ADM_ADM_3' read from 'gadm41_ESP.gpkg'.
Layer 'ADM_ADM_3' read from 'gadm41_FRA.gpkg'.
Layer 'ADM_ADM_3' read from 'gadm41_ITA.gpkg'.
Layer 'ADM_ADM_3' read from 'gadm41_LUX.gpkg'.
Layer 'ADM_ADM_3' read from 'gadm41_MWI.gpkg'.
Layer 'ADM_ADM_2' read from 'gadm41_NLD.gpkg' (fallback).
Layer 'ADM_ADM_3' read from 'gadm41_POL.gpkg'.
Layer 'ADM_ADM_2' read from 'gadm41_SVK.gpkg' (fallback).
Layer 'ADM_ADM_2' read from 'gadm41_SVN.gpkg' (fallback).
Layer 'ADM_ADM_2' read from 'gadm41_SWE.gpkg' (fallback).
Combined GeoDataFrame created.
File Deleted successfully


In [6]:
# Number of provinces per country

# Set your subfolder name here
subfolder_name = "data"
folder_path = os.path.join(os.getcwd(), subfolder_name)

# Dictionary to hold unique name1 values per country
provinces_counts = defaultdict(set)

# Iterate through each file in the subfolder
for filename in os.listdir(folder_path):
    if filename.endswith('.gpkg'):
        file_path = os.path.join(folder_path, filename)
        try:
            # List all layers in the GeoPackage
            layers = fiona.listlayers(file_path)
            if 'ADM_ADM_1' not in layers:
                print(f"Layer 'adm_1' not found in {filename}, skipping...")
                continue
            # Read the adm_1 layer
            gdf = gpd.read_file(file_path, layer='ADM_ADM_1')
            # Filter out rows where type1 is 'Water body'
            gdf_filtered = gdf[gdf['TYPE_1'] != 'Water body']
            # Group by country and collect unique name1 values
            for country, group in gdf_filtered.groupby('COUNTRY'):
                unique_name1s = group['NAME_1'].unique()
                provinces_counts[country].update(unique_name1s)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Convert the dictionary to a DataFrame
df_provinces = pd.DataFrame(
    [(country, len(name1_set)) for country, name1_set in provinces_counts.items()],
    columns=['country', 'provinces_total']
)

# Display the DataFrame
#print(df_provinces)

In [7]:
# Number of municipalities per country

# Dictionary to hold unique name1 values per country
municipalities_counts = defaultdict(set)

# Iterate through each file in the subfolder
for filename in os.listdir(folder_path):
    if filename.endswith('.gpkg'):
        file_path = os.path.join(folder_path, filename)
        try:
            # List all layers in the GeoPackage
            layers = fiona.listlayers(file_path)
            if 'ADM_ADM_2' not in layers:
                print(f"Layer 'adm_1' not found in {filename}, skipping...")
                continue
            # Read the adm_1 layer
            gdf = gpd.read_file(file_path, layer='ADM_ADM_2')
            # Filter out rows where type1 is 'Water body'
            gdf_filtered = gdf[gdf['TYPE_2'] != 'Water body']
            # Group by country and collect unique name1 values
            for country, group in gdf_filtered.groupby('COUNTRY'):
                # Create a combined key for uniqueness
                group['combined_key'] = group['NAME_1'].astype(str) + "_" + group['NAME_2'].astype(str)
                unique_combinations = group['combined_key'].unique()
                municipalities_counts[country].update(unique_combinations)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Convert the dictionary to a DataFrame
df_municipalities = pd.DataFrame(
    [(country, len(name1_set)) for country, name1_set in municipalities_counts.items()],
    columns=['country', 'muncipalities_total']
)

In [8]:
# Number of districs per country

# Dictionary to hold unique name1 values per country
districts_counts = defaultdict(set)

# Iterate through each file in the subfolder
for filename in os.listdir(folder_path):
    if filename.endswith('.gpkg'):
        file_path = os.path.join(folder_path, filename)
        try:
            # List all layers in the GeoPackage
            layers = fiona.listlayers(file_path)
            if 'ADM_ADM_3' not in layers:
                print(f"Layer 'adm_1' not found in {filename}, skipping...")
                continue
            # Read the adm_1 layer
            gdf = gpd.read_file(file_path, layer='ADM_ADM_3')
            # Filter out rows where type1 is 'Water body'
            gdf_filtered = gdf[gdf['TYPE_3'] != 'Water body']
            # Group by country and collect unique name1 values
            for country, group in gdf_filtered.groupby('COUNTRY'):
                # Create a combined key for uniqueness
                group['combined_key'] = group['NAME_1'].astype(str) + "_" + group['NAME_2'].astype(str) + "_" + group['NAME_3'].astype(str) 
                unique_combinations = group['combined_key'].unique()
                districts_counts[country].update(unique_combinations)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Convert the dictionary to a DataFrame
df_districts = pd.DataFrame(
    [(country, len(name1_set)) for country, name1_set in districts_counts.items()],
    columns=['country', 'districts_total']
)

Layer 'adm_1' not found in gadm41_CZE.gpkg, skipping...
Layer 'adm_1' not found in gadm41_DNK.gpkg, skipping...
Layer 'adm_1' not found in gadm41_NLD.gpkg, skipping...
Layer 'adm_1' not found in gadm41_SVK.gpkg, skipping...
Layer 'adm_1' not found in gadm41_SVN.gpkg, skipping...
Layer 'adm_1' not found in gadm41_SWE.gpkg, skipping...


In [9]:
# Merge total regions with regions run

provincies = gpd.read_file("adm_1.geojson")
# count number of provinces per country

#2. Convert to a DataFrame while naming the count column
count_provinces_run = (
    provincies
    .groupby(['country'])
    .size()
    .reset_index(name='provinces_run')
)

adm_1_stats = count_provinces_run.merge(df_provinces, on='country', how='inner')
#print(adm_1_stats)

#municipalities_run
municipalities = gpd.read_file("adm_2.geojson")
# count number of provinces per country

#2. Convert to a DataFrame while naming the count column
count_municipalities_run = (
    municipalities
    .groupby(['country'])
    .size()
    .reset_index(name='municipalities_run')
)

adm_2_stats = count_municipalities_run.merge(df_municipalities, on='country', how='inner')

#districts_run
districts = gpd.read_file("adm_3.geojson")
# count number of provinces per country

#2. Convert to a DataFrame while naming the count column
count_districts_run = (
    districts
    .groupby(['country'])
    .size()
    .reset_index(name='districts_run')
)

adm_3_stats = count_districts_run.merge(df_districts, on='country', how='inner')

# ----------------------------------------------------------------------
# WRITE OUTPUTS
# ----------------------------------------------------------------------
# JSON – easy for programs to consume

adm_1_stats.to_json('adm_1_stats.json', orient='records', lines=True)
adm_2_stats.to_json('adm_2_stats.json', orient='records', lines=True)
adm_3_stats.to_json('adm_3_stats.json', orient='records', lines=True)

In [10]:
# Useful statements

# import fiona
# layers = fiona.listlayers(file)

In [11]:
'''
# Move data from new_gpxfiles dir to processed_gpxfiles dir
# gather all files in source folder
allfiles = os.listdir(gpx_directory)
 
# iterate on all files to move them to destination folder
for f in allfiles:
    src_path = os.path.join(gpx_directory, f)
    dst_path = os.path.join(processed_gpx_directory, f)
    os.rename(src_path, dst_path)
    '''

'\n# Move data from new_gpxfiles dir to processed_gpxfiles dir\n# gather all files in source folder\nallfiles = os.listdir(gpx_directory)\n \n# iterate on all files to move them to destination folder\nfor f in allfiles:\n    src_path = os.path.join(gpx_directory, f)\n    dst_path = os.path.join(processed_gpx_directory, f)\n    os.rename(src_path, dst_path)\n    '